In [1]:
'''
<Transfer Learning>
In this script, we implement a simple transfer learning example for the task of classifying
cats and dogs using a trained model of InceptionV3 on ImageNet dataset. 
We remove the last FC layers of the InceptionV3 and attach a simple top model classifier. 

'''

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# set parameters here
# image size
img_width, img_height = 150, 150

# path to save model weights
top_model_weights_path = 'bottleneck_fc_model_inceptionv3.h5'
# path to train and validation data, note that each folder includes subfolders of subtypes, e.g. cat and dog
train_data_dir = 'C:\\Users\\sseyyedi\\Documents\\data\\all\\train'
validation_data_dir = 'C:\\Users\\sseyyedi\\Documents\\data\\all\\validation'
# total number of images for training
nb_train_samples = 2000
# total number of images for validation
nb_validation_samples = 800

epochs = 50
batch_size = 16

# to save teh bottleneck features from Inception v3
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG19 network
    model = applications.InceptionV3(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)

# to train the top model
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy','rb'))
    train_labels = np.array(
        [0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))
    validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
    validation_labels = np.array(
        [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

# RMSprop optimizer with a binary crossentropy
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    return history

Using TensorFlow backend.


In [2]:
# save the bottle-neck features and train top model
save_bottlebeck_features()
history = train_top_model()

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 5s 2ms/step - loss: 2.2350 - acc: 0.8435 - val_loss: 0.9642 - val_acc: 0.9325
Epoch 2/50
2000/2000 [==============================] - 2s 1ms/step - loss: 1.3161 - acc: 0.9100 - val_loss: 4.9780 - val_acc: 0.6763
Epoch 3/50
2000/2000 [==============================] - 2s 1ms/step - loss: 1.1360 - acc: 0.9220 - val_loss: 0.5987 - val_acc: 0.9600
Epoch 4/50
2000/2000 [==============================] - 2s 1ms/step - loss: 1.0428 - acc: 0.9315 - val_loss: 0.6752 - val_acc: 0.9513
Epoch 5/50
2000/2000 [==============================] - 2s 1ms/step - loss: 1.1561 - acc: 0.9230 - val_loss: 1.3722 - val_acc: 0.9125
Epoch 6/50
2000/2000 [==============================] - 2s 1ms/step - loss: 0.9918 - acc: 0.9325 - val_loss: 0.9602 - val_acc: 0.9375
Epoch 7/50
2000/2000 [==============================] - 2s 1ms/ste

In [ ]:
# plot accuracya nd loss plots for train and val data
import matplotlib.pyplot as plt
print(history.history.keys())
# accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()
# loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='center right')
plt.show()